In [1]:
import sys
import pandas as pd
from load_data import get_data, load_pkl, save_pkl
from processing_helpers import get_cue_segments, get_power_spec, get_peak_fits, get_control_segments
import pickle as pkl
import numpy as np
import os
import matplotlib.pyplot as plt
from fooof import FOOOF
from scipy.stats import sem, f_oneway, ttest_ind
import pingouin as pg
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import itertools
from statsmodels.formula.api import ols
import analysis_helpers as ah
from statsmodels.stats.multitest import multipletests


%matplotlib inline


In [3]:
# Global variables to store the session data
lfp_df = pd.DataFrame()
ch_num_list = []
lfp_mne = []
session_start_time = 0
markers = []
timeStamps = []
sampling_frequency = 0
cue_segments_df = pd.DataFrame()
control_segments_df = pd.DataFrame()

theta_range = [3, 12]

In [16]:
def load_data(day):
    """
    Load LFP data for a given day.
    """
    global lfp_df, ch_num_list, lfp_mne, session_start_time, markers, timeStamps, sampling_frequency
    print(f"Loading data for {day}...")
    lfp_df, ch_num_list, lfp_mne, session_start_time, markers, timeStamps, sampling_frequency = get_data(day)

    if lfp_df is not None:
        print(f"Loaded {len(ch_num_list)} channels.")
    else:
        print("Failed to load data.")

def compute_theta_trials(data, key='flat_spec_og'):
    """
    For each trial in the DataFrame, compute the sum of power in the theta band.
    """
    if data.empty:
        return []
    psd_list = data[key].tolist()
    lengths = [arr.shape[0] for arr in psd_list if hasattr(arr, 'shape')]
    if not lengths:
        return []
    min_length = min(lengths)
    # Compute the theta power by summing values in indices corresponding to theta_range
    trial_values = [np.sum(psd[:min_length][theta_range[0]:theta_range[1]]) for psd in psd_list]
    return trial_values

def compute_venn_counts(result_df, factors, alpha=0.05):
    """
    Returns a dict of counts in each of the 8 Venn regions,
    but treats each row of result_df as its own element
    (so repeated channel numbers on different days stay distinct).

    Expects result_df to have columns:
       factors[0], factors[1], f"{factors[0]}:{factors[1]}", and 'Channel'
    """
    f1, f2 = factors
    p1    = f1
    p2    = f2
    pint  = f"{f1}:{f2}"

    # use the DataFrame index as the “universe” of unique items
    all_ids = set(result_df.index)

    # find which rows are significant for each effect
    sig1 = set(result_df.index[result_df[p1]   < alpha])
    sig2 = set(result_df.index[result_df[p2]   < alpha])
    sig3 = set(result_df.index[result_df[pint] < alpha])

    union123 = sig1 | sig2 | sig3

    return {
        f"{f1}_only":       len(sig1   - sig2     - sig3),
        f"{f2}_only":       len(sig2   - sig1     - sig3),
        "interaction_only": len(sig3   - sig1     - sig2),
        f"{f1}_{f2}_only":  len((sig1 & sig2)     - sig3),
        f"{f1}_int_only":   len((sig1 & sig3)     - sig2),
        f"{f2}_int_only":   len((sig2 & sig3)     - sig1),
        "all_three":        len(sig1 & sig2 & sig3),
        "none":             len(all_ids - union123),
    }


def map_to_pillar(value, mapping):
    """
    Map a given numeric value to a pillar group index based on a provided mapping list.
    """
    for i, group in enumerate(mapping):
        if value in group:
            return i
    return np.nan

def safe_max(x):
    if not x:
        # empty → leave empty
        return np.nan
    # if x is a nested list (e.g. [[...]]), flatten it
    flat = np.concatenate(x) if any(isinstance(el, (list, np.ndarray)) for el in x) else np.array(x)
    return flat.max().item()


In [18]:
# Load Raw Data
days = ['20181102', '20181105', '20181101', '20181026', '20181022']
for day in days:
    load_data(day)

    # Extract segments with a specific window size and process
    window_size = 1000  # e.g., 1000 ms
    peak_fit_range = [1,14]

    interesting_ch = {
        '20181102' : [9,19,26,29,30,31,43,45],
        '20181105' : [19,23,29,30,43,45],
        '20181101' : [19,21,23,29,30,35,43,45],
        '20181031' : [19,26,29,30,35,43,45],
        '20181026' : [19,29,30,35,45],
        '20181022' : [9,17,26,29,43,45]
    }

    cue_segments_df = load_pkl(f"/Users/liuyuanwei/Desktop/Data Processed/{day}/{day}_cue.pkl")
    control_segments_df = load_pkl(f"/Users/liuyuanwei/Desktop/Data Processed/{day}/{day}_control.pkl")


    channel_no = sorted(cue_segments_df['channel'].unique())
    ch_indx = []
    for i in interesting_ch[day]:
        ch_str = f"{i:03}"
        if ch_str in channel_no:
            ch_indx.append(channel_no.index(ch_str))

    # --- Preprocess: Convert channel values to zero-padded strings ---
    cue_segments_df['channel'] = cue_segments_df['channel'].apply(lambda x: f"{int(x):03}")
    
    cue_segments_df['theta_power'] = cue_segments_df['flat_spec_og'].apply(lambda x: np.sum(x[theta_range[0]:theta_range[1]]))
    cue_segments_df['peak_gauss_max'] = cue_segments_df['peak_gauss'].apply(safe_max)

    # Define pillar groupings and mapping for colors
    pillar_destination = [[11, 16], [12], [13], [14, 15]]
    pillar_start = [[31, 36], [32], [33], [34, 35]]

    pillar_color_map = {
        0: 'blue',    # Pillar 1: Posters 1 & 6
        1: 'yellow',  # Pillar 2: Poster 2
        2: 'green',   # Pillar 3: Poster 3
        3: 'red'      # Pillar 4: Posters 4 & 5
    }


    # Create new columns for pillar start and pillar destination.
    cue_segments_df['pillar_start'] = cue_segments_df['start_position'].apply(lambda x: map_to_pillar(x, pillar_start))
    cue_segments_df['pillar_destination'] = cue_segments_df['cue_onset'].apply(lambda x: map_to_pillar(x, pillar_destination))
    


    # ————————————————————————————————————————————————————————————————
    output_base = f"/Users/liuyuanwei/Documents/GitHub/FYP-Pyhipp/LFP_Analysis/Figures/{day}"

    jobs = [
        # ("theta_power", "Theta_Power",
        #     [("start_position","cue_onset","StartPoster_CuePoster_results"),
        #     ("pillar_start",  "pillar_destination","StartPillar_CuePillar_results"),
        #     ]),
        ("peak_gauss_max",  "Theta_Peak_Power",
            [("start_position","cue_onset","StartPoster_CuePoster_results"),
            ("pillar_start",  "pillar_destination","StartPillar_CuePillar_results"),
            ]),
    ]

    channels = [int(x) for x in sorted(cue_segments_df['channel'].unique())]

    for dependent, main_folder, analyses in jobs:
        for f1, f2, subdir in analyses:
            out_dir = os.path.join(output_base, main_folder, subdir)

            # 1) run the pipeline
            result_df = ah.run_analysis_pipeline(
                cue_segments_df, channels,
                dependent=dependent,
                factors=[f1, f2],
                plot_groups=(f1, f2),
                output_folder=out_dir,
                alpha=0.05
            )

            # 2) compute the counts
            counts = compute_venn_counts(result_df, (f1, f2), alpha=0.05)

            # 3) dump to CSV
            os.makedirs(out_dir, exist_ok=True)
            row = {
                "dependent": dependent,
                "factor1":   f1,
                "factor2":   f2,
                **counts
            }
            df_counts = pd.DataFrame([row])
            csv_path = os.path.join(out_dir, "venn_counts.csv")
            df_counts.to_csv(csv_path, index=False)

            print(f"Written counts for {dependent} [{f1} vs {f2}] to {csv_path}")


Loading data for 20181102...
Creating RawArray with float64 data, n_channels=26, n_times=6316854
    Range : 0 ... 6316853 =      0.000 ...  6316.853 secs
Ready.
Loaded 26 channels.
Data successfully loaded from /Users/liuyuanwei/Desktop/Data Processed/20181102/20181102_cue.pkl
Loaded data has 10374 rows and 14 columns.
                                             segment channel start_position  \
0  [8.966168, 9.071849, 9.07477, 8.924825, 7.6476...     009             33   
1  [18.236647, 25.963833, 28.956192, 31.728855, 3...     019             33   
2  [78.45129, 69.64043, 59.739246, 53.65224, 53.8...     021             33   
3  [-20.94718, -18.200512, -14.654313, -10.096560...     022             33   
4  [43.646423, 47.14898, 51.073883, 57.155018, 65...     023             33   

  cue_onset                                                psd  \
0        11  [1.904857606414403, 2.6051776546205625, 1.6806...   
1        11  [1.6660689893954277, 2.038724929269758, 1.4047...   
2    

/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 25, but rank is 20
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The numb

Written counts for peak_gauss_max [start_position vs cue_onset] to /Users/liuyuanwei/Documents/GitHub/FYP-Pyhipp/LFP_Analysis/Figures/20181102/Theta_Peak_Power/StartPoster_CuePoster_results/venn_counts.csv


/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 7
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number

Written counts for peak_gauss_max [pillar_start vs pillar_destination] to /Users/liuyuanwei/Documents/GitHub/FYP-Pyhipp/LFP_Analysis/Figures/20181102/Theta_Peak_Power/StartPillar_CuePillar_results/venn_counts.csv
Loading data for 20181105...
Creating RawArray with float64 data, n_channels=26, n_times=5496020
    Range : 0 ... 5496019 =      0.000 ...  5496.019 secs
Ready.
Loaded 26 channels.
Data successfully loaded from /Users/liuyuanwei/Desktop/Data Processed/20181105/20181105_cue.pkl
Loaded data has 10374 rows and 14 columns.
                                             segment channel start_position  \
0  [-4.5271883, -1.9674453, 9.442836, 25.153576, ...     009             34   
1  [-42.301697, -45.33196, -43.950962, -39.873737...     019             34   
2  [-76.19089, -30.659563, 20.823097, 73.89416, 1...     021             34   
3  [-80.23443, -129.71568, -178.99425, -225.3264,...     022             34   
4  [-4.759627, -4.6272993, -2.6809525, 1.0050095,...     023          

...

Written counts for peak_gauss_max [pillar_start vs pillar_destination] to /Users/liuyuanwei/Documents/GitHub/FYP-Pyhipp/LFP_Analysis/Figures/20181026/Theta_Peak_Power/StartPillar_CuePillar_results/venn_counts.csv
Loading data for 20181022...
Creating RawArray with float64 data, n_channels=26, n_times=4942248
    Range : 0 ... 4942247 =      0.000 ...  4942.247 secs
Ready.
Loaded 26 channels.
Data successfully loaded from /Users/liuyuanwei/Desktop/Data Processed/20181022/20181022_cue.pkl
Loaded data has 10374 rows and 14 columns.
                                             segment channel start_position  \
0  [9.893615, 7.5915875, 3.9290628, 0.5271555, -1...     009             36   
1  [4.7024913, 2.796567, 1.1272492, 1.355152, 4.2...     011             36   
2  [19.14239, 17.051285, 15.416498, 15.21809, 16....     015             36   
3  [4.157256, 3.296375, 2.3582096, 1.762686, 2.12...     017             36   
4  [4.98611, 4.4083276, 3.8366354, 3.646438, 4.25...     020          

/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 25, but rank is 20
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The numb

Written counts for peak_gauss_max [start_position vs cue_onset] to /Users/liuyuanwei/Documents/GitHub/FYP-Pyhipp/LFP_Analysis/Figures/20181022/Theta_Peak_Power/StartPoster_CuePoster_results/venn_counts.csv


/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 7
  warnings.warn('covariance of constraints does not have full '
/Users/liuyuanwei/anaconda3/envs/pyhipp/lib/python3.8/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number

Written counts for peak_gauss_max [pillar_start vs pillar_destination] to /Users/liuyuanwei/Documents/GitHub/FYP-Pyhipp/LFP_Analysis/Figures/20181022/Theta_Peak_Power/StartPillar_CuePillar_results/venn_counts.csv


In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# Ensure directories exist
results_dir = "Figures/theta_results"
plots_dir = "Figures/theta_plots"
os.makedirs(results_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

days = ['20181102', '20181105', '20181101', '20181026', '20181022']

for day in days:
    # Load data for the day
    load_data(day)
    cue_segments_df = load_pkl(f"/Users/liuyuanwei/Desktop/Data Processed/{day}/{day}_cue.pkl")
    control_segments_df = load_pkl(f"/Users/liuyuanwei/Desktop/Data Processed/{day}/{day}_control.pkl")

    # Preprocess channel strings and compute theta power
    cue_segments_df['channel'] = cue_segments_df['channel'].apply(lambda x: f"{int(x):03}")
    theta_range = [3, 12]
    
    cue_segments_df['theta_power'] = cue_segments_df['flat_spec_og'].apply(lambda x: np.sum(x[theta_range[0]:theta_range[1]]))
    control_segments_df['theta_power'] = control_segments_df['flat_spec_og'].apply(lambda x: np.sum(x[theta_range[0]:theta_range[1]]))

    # Identify channels present in both conditions
    channels = sorted(set(control_segments_df['channel']).intersection(cue_segments_df['channel']))

    # Perform t-tests for each channel
    results = []
    for ch in channels:
        ctrl = control_segments_df.loc[control_segments_df['channel'] == ch, 'theta_power']
        cue  = cue_segments_df.loc[cue_segments_df['channel'] == ch, 'theta_power']
        stat, pval = ttest_ind(ctrl, cue, equal_var=False, nan_policy='omit')
        results.append((ch, pval))

    chs, pvals = zip(*results)
    reject, pvals_corr, _, _ = multipletests(pvals, alpha=0.05, method='bonferroni')

    # Create summary DataFrame and save
    summary_df = pd.DataFrame({
        'channel': chs,
        f'p_corrected_{day}': pvals_corr,
        f'significant_{day}': reject
    })
    csv_filename = os.path.join(results_dir, f"theta_power_results_{day}.csv")
    summary_df.to_csv(csv_filename, index=False)

    # Prepare data for plotting
    ctrl_df = control_segments_df.assign(condition='control')
    cue_df  = cue_segments_df.assign(condition='cue')
    combined = pd.concat([ctrl_df, cue_df])

    # Extract per-channel data lists
    
    ctrl_data = [combined[(combined.channel == ch) & (combined.condition == 'control')]['theta_power'] for ch in channels]
    cue_data  = [combined[(combined.channel == ch) & (combined.condition == 'cue')]['theta_power'] for ch in channels]

    # Plot configuration
    plt.figure(figsize=(12, 6))
    x = np.arange(len(channels))
    width = 0.35

    # Draw boxplots
    bp1 = plt.boxplot(ctrl_data, positions=x - width/2, widths=width, patch_artist=True,
                      boxprops=dict(facecolor="lightblue"))
    bp2 = plt.boxplot(cue_data,  positions=x + width/2, widths=width, patch_artist=True,
                      boxprops=dict(facecolor="orange"))

    # Compute fixed marker height
    all_vals = np.concatenate([np.concatenate(ctrl_data), np.concatenate(cue_data)])
    y_min, y_max = np.nanmin(all_vals), np.nanmax(all_vals)
    y_marker = y_max + (y_max - y_min) * 0.05

    # Annotate significant channels at same height
    for i, is_sig in enumerate(reject):
        if is_sig:
            plt.text(x[i], y_marker, '[*]', ha='center', va='bottom', fontsize=14)

    # Labels and legend
    plt.xticks(x, channels)
    plt.xlabel('Channel')
    plt.ylabel('Theta Power')
    plt.title(f'Control vs Cue Theta Power by Channel ({day})')
    plt.legend([bp1['boxes'][0], bp2['boxes'][0]], ['Control', 'Cue'])
    plt.ylim(bottom=y_min - (y_max - y_min) * 0.05, top=y_marker + (y_max - y_min) * 0.05)
    plt.tight_layout()

    # Save plot
    plot_filename = os.path.join(plots_dir, f"theta_power_{day}.png")
    plt.savefig(plot_filename)
    plt.close()

print("Results and plots saved successfully, with bracketed asterisks marking significance at a constant height.")


Loading data for 20181102...
Creating RawArray with float64 data, n_channels=26, n_times=6316854
    Range : 0 ... 6316853 =      0.000 ...  6316.853 secs
Ready.
Loaded 26 channels.
  channel                                           lfp_data
0     009  [12.966919, 9.920207, 7.1469517, 5.445115, 6.0...
1     019  [-2.9494066, -4.854547, -2.579513, 5.369741, 1...
2     021  [-25.190666, -62.702038, -91.88798, -108.15420...
3     022  [2.2200575, 26.696917, 43.72904, 50.191273, 48...
4     023  [-4.953173, -20.97119, -33.501663, -40.505997,...
Data successfully loaded from /Users/liuyuanwei/Desktop/Data Processed/20181102/20181102_cue.pkl
Loaded data has 10374 rows and 14 columns.
                                             segment channel start_position  \
0  [8.966168, 9.071849, 9.07477, 8.924825, 7.6476...     009             33   
1  [18.236647, 25.963833, 28.956192, 31.728855, 3...     019             33   
2  [78.45129, 69.64043, 59.739246, 53.65224, 53.8...     021             33